# Fashion Recommendation RAG System

This notebook implements a LangChain-powered RAG system for fashion recommendations using fine-tuned CLIP embeddings.

## Overview
Given a natural-language query (e.g., 'What would look good on a brunch date?'), the system:
1. Embeds the query using our fine-tuned CLIP model
2. Retrieves top-k fashion items from ChromaDB vector database  
3. Uses LangChain to generate personalized fashion advice

## Architecture
- **Vector Database**: ChromaDB with fashion product embeddings and metadata
- **Retriever**: Custom fashion product retriever with metadata filtering
- **LLM**: GPT-4.1-mini for generating fashion advice
- **State Management**: LangGraph for handling query processing workflow

In [18]:


!pip install langchain langchain-openai langchain-community langchain-huggingface chromadb sentence-transformers langgraph"



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import getpass
import os
import warnings
warnings.filterwarnings('ignore')

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4.1-mini", model_provider="openai")

In [20]:
import pandas as pd
import numpy as np
import chromadb
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings
from PIL import Image
import torch
from typing import List, Dict, Any, Optional
import json
from pathlib import Path

# Load our fine-tuned CLIP model for fashion embeddings using HuggingFace
model_name = "dejasi5459/clip-fashion-embeddings-final-10k-ft"
try:
    # Initialize HuggingFace embeddings with the fine-tuned model
    fashion_embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )
    
    # Also load SentenceTransformer for backward compatibility with existing retriever
    fashion_model = SentenceTransformer(model_name)
    print("Loaded fine-tuned fashion CLIP model successfully with HuggingFace integration")
except Exception as e:
    print(f"Error loading fine-tuned model: {e}")
    # Fallback to base model if fine-tuned model not available
    fashion_embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/clip-ViT-L-14',
        model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )
    fashion_model = SentenceTransformer('sentence-transformers/clip-ViT-L-14')
    print("Using base CLIP model (fine-tuned model not found)")

Loaded fine-tuned fashion CLIP model successfully with HuggingFace integration


In [21]:
# Initialize ChromaDB client and connect to fashion embeddings collection
chroma_client = chromadb.PersistentClient(path="./fashion_vector_db")

# Try to get existing collection
try:
    fashion_collection = chroma_client.get_collection("fashion_products")
    print(f"Connected to existing fashion collection with {fashion_collection.count()} products")
except:
    print("Fashion embeddings collection not found. Please run embeddings_generation.ipynb first.")
    print("Creating empty collection for demonstration...")
    fashion_collection = chroma_client.create_collection(
        name="fashion_products",
        metadata={"description": "Fashion product embeddings with metadata"}
    )

Connected to existing fashion collection with 88838 products


In [22]:
from langchain.schema import Document

class FashionProductRetriever:
    """Enhanced retriever for fashion products with metadata filtering and LangChain integration."""
    
    def __init__(self, collection, model, langchain_vectorstore=None, top_k: int = 5):
        self.collection = collection
        self.model = model
        self.langchain_vectorstore = langchain_vectorstore
        self.top_k = top_k
    
    def search(self, query: str, filters: Optional[Dict] = None) -> List[Dict]:
        """Search for fashion products based on natural language query."""
        
        # Generate embedding for the query
        query_embedding = self.model.encode([query]).tolist()
        
        # Build ChromaDB where clause for filters
        where_clause = None
        if filters:
            conditions = []
            
            # Handle price range separately
            if 'price_range' in filters:
                min_price, max_price = filters.pop('price_range')
                conditions.append({"price": {"$gte": min_price}})
                conditions.append({"price": {"$lte": max_price}})

            # Handle other equality filters
            if filters:
                conditions.extend([{key: {"$eq": value}} for key, value in filters.items()])

            if len(conditions) > 1:
                where_clause = {"$and": conditions}
            elif len(conditions) == 1:
                where_clause = conditions[0]
        
        # Query the collection
        try:
            results = self.collection.query(
                query_embeddings=query_embedding,
                n_results=self.top_k,
                where=where_clause,
                include=["metadatas", "documents", "distances"]
            )
            
            # Format results
            formatted_results = []
            if results['documents'] and results['documents'][0]:
                for i in range(len(results['documents'][0])):
                    formatted_results.append({
                        'id': results['ids'][0][i],
                        'product_name': results['documents'][0][i],
                        'metadata': results['metadatas'][0][i] if results['metadatas'] else {},
                        'similarity': 1 - results['distances'][0][i]  # Convert distance to similarity
                    })
            
            return formatted_results
            
        except Exception as e:
            print(f"Search error: {e}")
            return []
    
    def search_with_langchain(self, query: str, k: int = None) -> List[Document]:
        """Search using LangChain vector store interface."""
        if not self.langchain_vectorstore:
            print("LangChain vector store not available")
            return []
        
        try:
            k = k or self.top_k
            docs = self.langchain_vectorstore.similarity_search(query, k=k)
            return docs
        except Exception as e:
            print(f"LangChain search error: {e}")
            return []
    
    def get_product_details(self, product_ids: List[str]) -> List[Dict]:
        """Get detailed information for specific product IDs."""
        try:
            results = self.collection.get(ids=product_ids)
            
            formatted_results = []
            for i, prod_id in enumerate(results['ids']):
                formatted_results.append({
                    'id': prod_id,
                    'product_name': results['documents'][i],
                    'metadata': results['metadatas'][i] if results['metadatas'] else {}
                })
            
            return formatted_results
            
        except Exception as e:
            print(f"Error getting product details: {e}")
            return []

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma

# Create LangChain vector store using our existing ChromaDB collection and HuggingFace embeddings
# This provides a LangChain-compatible interface to our fashion embeddings
try:
    langchain_vectorstore = Chroma(
        client=chroma_client,
        collection_name="fashion_products",
        embedding_function=fashion_embeddings
    )
    print("LangChain vector store initialized successfully with HuggingFace embeddings")
    
    # Test the vector store with a sample query
    test_docs = langchain_vectorstore.similarity_search("casual shirt", k=3)
    print(f"Vector store test: Found {len(test_docs)} similar products")
    
except Exception as e:
    print(f"Error initializing LangChain vector store: {e}")
    langchain_vectorstore = None

# Initialize the enhanced fashion retriever
fashion_retriever = FashionProductRetriever(
    fashion_collection, 
    fashion_model, 
    langchain_vectorstore,
    top_k=8
)

from langgraph.graph import START, StateGraph, END
from typing_extensions import List, TypedDict

# Define state for our fashion RAG workflow
class FashionRAGState(TypedDict):
    """State for fashion recommendation workflow."""
    query: str
    user_preferences: Optional[Dict]
    retrieved_products: List[Dict]
    recommendation: str
    explanation: str

# Fashion-specific prompt template
fashion_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a professional fashion stylist and advisor. Your role is to provide personalized fashion recommendations based on retrieved product information.

Given a user's query and a list of fashion products, provide thoughtful styling advice that considers:
- The occasion or context mentioned in the query
- The user's preferences (if provided)
- How the retrieved items can be styled together
- Fashion principles like color coordination, style coherence, and appropriateness

Be specific about why each recommended item works for the given context and how they can be combined for a complete look."""),
    
    ("human", """User Query: {query}

User Preferences: {preferences}

Retrieved Fashion Products:
{products}

Please provide a detailed fashion recommendation with explanation.""")
])

def format_products_for_prompt(products: List[Dict]) -> str:
    """Format retrieved products for the prompt."""
    if not products:
        return "No products found matching the criteria."
    
    formatted = []
    for i, product in enumerate(products, 1):
        metadata = product.get('metadata', {})
        
        product_info = [
            f"{i}. {product.get('product_name', 'Unknown Product')}"
        ]
        
        # Add available metadata
        if metadata.get('gender'):
            product_info.append(f"   - Gender: {metadata['gender']}")
        if metadata.get('masterCategory'):
            product_info.append(f"   - Category: {metadata['masterCategory']}")
        if metadata.get('subCategory'):
            product_info.append(f"   - Type: {metadata['subCategory']}")
        if metadata.get('articleType'):
            product_info.append(f"   - Article: {metadata['articleType']}")
        if metadata.get('baseColour'):
            product_info.append(f"   - Color: {metadata['baseColour']}")
        if metadata.get('season'):
            product_info.append(f"   - Season: {metadata['season']}")
        if metadata.get('usage'):
            product_info.append(f"   - Usage: {metadata['usage']}")
        if metadata.get('price_range'):
            product_info.append(f"   - Price Range: {metadata['price_range']}")
        
        similarity = product.get('similarity', 0)
        product_info.append(f"   - Relevance Score: {similarity:.3f}")
        
        formatted.append('\n'.join(product_info))
    
    return '\n\n'.join(formatted)

LangChain vector store initialized successfully with HuggingFace embeddings
Vector store test: Found 3 similar products
Vector store test: Found 3 similar products


In [24]:
def retrieve(state: FashionRAGState) -> FashionRAGState:
    """Retrieve relevant fashion products based on user query and preferences."""
    
    query = state["query"]
    user_preferences = state.get("user_preferences", {})
    
    print(f" Retrieving products for query: '{query}'")
    
    # Extract filters from user preferences
    filters = {}
    if user_preferences:
        if user_preferences.get("gender"):
            filters["gender"] = user_preferences["gender"]
        if user_preferences.get("style"):
            # Map style preferences to categories if needed
            style_mapping = {
                "casual": "Casual",
                "formal": "Formal", 
                "sporty": "Sports"
            }
            if user_preferences["style"] in style_mapping:
                filters["usage"] = style_mapping[user_preferences["style"]]
        if user_preferences.get("category"):
            filters["masterCategory"] = user_preferences["category"]
    
    # Search for products
    retrieved_products = fashion_retriever.search(query, filters)
    
    print(f"Retrieved {len(retrieved_products)} products")
    
    return {
        "query": query,
        "user_preferences": user_preferences,
        "retrieved_products": retrieved_products,
        "recommendation": "",
        "explanation": ""
    }

In [25]:
def generate(state: FashionRAGState) -> FashionRAGState:
    """Generate fashion recommendations using retrieved products."""
    
    query = state["query"]
    user_preferences = state.get("user_preferences", {})
    retrieved_products = state["retrieved_products"]
    
    print(f"Generating fashion advice...")
    
    # Format products for the prompt
    products_text = format_products_for_prompt(retrieved_products)
    preferences_text = json.dumps(user_preferences, indent=2) if user_preferences else "No specific preferences provided"
    
    # Create the prompt
    prompt = fashion_prompt.format_messages(
        query=query,
        preferences=preferences_text,
        products=products_text
    )
    
    # Generate response
    try:
        response = llm.invoke(prompt)
        recommendation = response.content
        
        # Extract explanation (for now, use the full response as explanation)
        explanation = f"Based on your query '{query}' and the retrieved products, here's my fashion advice."
        
    except Exception as e:
        print(f"Error generating recommendation: {e}")
        recommendation = "I apologize, but I'm having trouble generating a recommendation right now. Please try again."
        explanation = "Error occurred during generation."
    
    return {
        "query": query,
        "user_preferences": user_preferences,
        "retrieved_products": retrieved_products,
        "recommendation": recommendation,
        "explanation": explanation
    }

In [26]:
# Build the fashion RAG workflow graph
workflow = StateGraph(FashionRAGState)

# Add nodes
workflow.add_node("retrieve", retrieve)
workflow.add_node("generate", generate)

# Add edges
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

# Compile the graph
fashion_rag_app = workflow.compile()

print("Fashion RAG workflow compiled successfully!")

Fashion RAG workflow compiled successfully!


In [27]:
def get_fashion_advice(query: str, user_preferences: Optional[Dict] = None) -> Dict:
    """
    Get fashion advice for a natural language query.
    
    Args:
        query: Natural language fashion query (e.g., "What should I wear for a brunch date?")
        user_preferences: Optional user preferences dict with keys like:
            - gender: "Men", "Women", "Boys", "Girls", "Unisex"
            - style: "casual", "formal", "sporty"
            - category: "Apparel", "Accessories", "Footwear"
            - price_range: [min, max]
    
    Returns:
        Dict with recommendation, explanation, and retrieved products
    """
    
    # Create initial state
    initial_state = {
        "query": query,
        "user_preferences": user_preferences or {},
        "retrieved_products": [],
        "recommendation": "",
        "explanation": ""
    }
    
    # Run the workflow
    result = fashion_rag_app.invoke(initial_state)
    
    # Format the response
    return {
        "query": result["query"],
        "items": result["retrieved_products"],
        "recommendation": result["recommendation"],
        "explanation": result["explanation"],
        "user_preferences": result["user_preferences"]
    }

def display_fashion_advice(result: Dict):
    """Display fashion advice in a formatted way."""
    
    print("=" * 60)
    print(" FASHION RECOMMENDATION")
    print("=" * 60)
    print(f"Query: {result['query']}")
    
    if result['user_preferences']:
        print(f"Preferences: {result['user_preferences']}")
    
    print("\nRetrieved Products:")
    for i, item in enumerate(result['items'], 1):
        metadata = item.get('metadata', {})
        print(f"  {i}. {item.get('product_name', 'Unknown')}")
        if metadata.get('articleType'):
            print(f"     Type: {metadata['articleType']}")
        if metadata.get('baseColour'):
            print(f"     Color: {metadata['baseColour']}")
        if metadata.get('price_range'):
            print(f"     Price: {metadata['price_range']}")
    
    print(f"\n Recommendation:")
    print(result['recommendation'])
    print("=" * 60)

# Test HuggingFace embeddings integration
print("\nTesting HuggingFace Embeddings Integration")
print("=" * 50)

# Test embedding generation
test_query = "casual blue shirt"
test_embedding = fashion_embeddings.embed_query(test_query)
print(f"Query: '{test_query}'")
print(f"Embedding dimension: {len(test_embedding)}")
print(f"Embedding type: {type(test_embedding)}")

# Test LangChain vector store search
if langchain_vectorstore:
    print(f"\nTesting LangChain vector store search...")
    langchain_results = fashion_retriever.search_with_langchain(test_query, k=3)
    print(f"Found {len(langchain_results)} results via LangChain")
    for i, doc in enumerate(langchain_results[:2], 1):
        print(f"  {i}. {doc.page_content}")
        if hasattr(doc, 'metadata') and doc.metadata:
            print(f"     Metadata: {doc.metadata}")

print("HuggingFace integration test completed")


Testing HuggingFace Embeddings Integration
Query: 'casual blue shirt'
Embedding dimension: 768
Embedding type: <class 'list'>

Testing LangChain vector store search...
Found 3 results via LangChain
  1. Basics Men Blue Shirt | Gender: Men | Category: Apparel - Topwear | Type: Shirts | Color: Blue | Season: Summer | Usage: Casual | Year: 2012
     Metadata: {'price': 500.0, 'description': 'Basics Men Blue Shirt | Gender: Men | Category: Apparel - Topwear | Type: Shirts | Color: Blue | Season: Summer | Usage: Casual | Year: 2012', 'season': 'Summer', 'article_type': 'Shirts', 'image_path': 'fashion-product-images-small/images\\41588.jpg', 'year': 2012, 'gender': 'Men', 'base_colour': 'Blue', 'product_name': 'Basics Men Blue Shirt', 'usage': 'Casual', 'embedding_type': 'text', 'sub_category': 'Topwear', 'master_category': 'Apparel', 'product_id': '41588'}
  2. Basics Men Blue Shirt | Gender: Men | Category: Apparel - Topwear | Type: Shirts | Color: Blue | Season: Summer | Usage: Casual |

## Example Usage

Let's test our fashion RAG system with various queries and user preferences.

In [28]:
# Example 1: Basic query without specific preferences
query1 = "What would look good on my sister's wedding on my sister, we are planning a traditional south indian marriage ceremony?"

print("Example 1: Brunch Date Outfit ")
print("-" * 40)

result1 = get_fashion_advice(query1)
display_fashion_advice(result1)

Example 1: Brunch Date Outfit 
----------------------------------------
 Retrieving products for query: 'What would look good on my sister's wedding on my sister, we are planning a traditional south indian marriage ceremony?'
Retrieved 8 products
Generating fashion advice...
 FASHION RECOMMENDATION
Query: What would look good on my sister's wedding on my sister, we are planning a traditional south indian marriage ceremony?

Retrieved Products:
  1. FNF Green Collection for Wedding Sari | Gender: Women | Category: Apparel - Saree | Type: Sarees | Color: Green | Season: Fall | Usage: Ethnic | Year: 2012
  2. FNF Green Collection for Wedding Sari | Gender: Women | Category: Apparel - Saree | Type: Sarees | Color: Green | Season: Fall | Usage: Ethnic | Year: 2012
  3. Image: FNF Maroon Printed Wedding Collection Sari
  4. Image: Fabindia Maroon Mangalgiri Saree
  5. Image: Fabindia Pink Mangalagiri Saree
  6. Image: FNF Red Bridal Collection Lehenga Choli
  7. Image: Fabindia Green Mangala

In [29]:
# Example 2: Query with user preferences
query2 = "I need something professional for a business meeting"
preferences2 = {
    "gender": "Women",
    "style": "formal",
    "category": "Apparel"
}

print("\nExample 2: Business Meeting Outfit")
print("-" * 40)

result2 = get_fashion_advice(query2, preferences2)
display_fashion_advice(result2)


Example 2: Business Meeting Outfit
----------------------------------------
 Retrieving products for query: 'I need something professional for a business meeting'
Retrieved 0 products
Generating fashion advice...
Retrieved 0 products
Generating fashion advice...
 FASHION RECOMMENDATION
Query: I need something professional for a business meeting
Preferences: {'gender': 'Women', 'style': 'formal', 'category': 'Apparel'}

Retrieved Products:

 Recommendation:
For a professional business meeting, especially with a preference for formal women's apparel, the key is to balance a polished, authoritative look with comfort and confidence. Here’s a detailed recommendation to help you dress appropriately:

1. Tailored Blazer and Trousers or Skirt:
- Choose a well-fitted blazer in a neutral color such as navy, black, charcoal, or deep gray. These colors convey professionalism and are versatile.
- Pair this blazer with matching tailored trousers or a pencil skirt. The skirt should be knee-length fo

In [30]:
# Example 3: Casual weekend outfit
query3 = "What should I wear for a casual weekend shopping trip?"
preferences3 = {
    "gender": "Men",
    "style": "casual"
}

print("\nExample 3: Casual Weekend Outfit")
print("-" * 40)

result3 = get_fashion_advice(query3, preferences3)
display_fashion_advice(result3)


Example 3: Casual Weekend Outfit
----------------------------------------
 Retrieving products for query: 'What should I wear for a casual weekend shopping trip?'
Retrieved 8 products
Generating fashion advice...
Retrieved 8 products
Generating fashion advice...
 FASHION RECOMMENDATION
Query: What should I wear for a casual weekend shopping trip?
Preferences: {'gender': 'Men', 'style': 'casual'}

Retrieved Products:
  1. Image: Myntra Men Yellow Check Shorts
  2. Image: Myntra Men Blue Check Shorts
  3. Image: Spykar Men Blue Jeans
  4. Image: Deni Yo Men Washed Blue Jeans
  5. Image: Deni Yo Men Washed Blue Jeans
  6. Image: Wills Lifestyle Men Cream Trousers
  7. Image: French Connection Men Black Check Shirt
  8. Image: Spykar Men Skinny Low Waist Tight Fit Blue Jeans

 Recommendation:
For a casual weekend shopping trip, comfort and ease of movement are key, while still looking put-together and stylish. Based on your preference for men's casual style and the retrieved products, her

In [31]:
# Interactive Testing
print("\n Interactive Fashion Advice System")
print("=" * 50)
print("Try your own queries! Examples:")
print("- 'What should I wear for a first date?'")
print("- 'I need summer clothes for vacation'")
print("- 'What goes well with black jeans?'")
print("- 'Suggest something trendy and comfortable'")

# Uncomment and modify the lines below to test with your own queries
# custom_query = "Your query here"
# custom_preferences = {"gender": "Women", "style": "casual"}  # Optional
# custom_result = get_fashion_advice(custom_query, custom_preferences)
# display_fashion_advice(custom_result)


 Interactive Fashion Advice System
Try your own queries! Examples:
- 'What should I wear for a first date?'
- 'I need summer clothes for vacation'
- 'What goes well with black jeans?'
- 'Suggest something trendy and comfortable'


## System Evaluation & Analysis

The fashion RAG system demonstrates the following capabilities:

### Key Features
1. **Multi-modal Embeddings**: Uses fine-tuned CLIP model for fashion-specific representations
2. **Metadata Filtering**: Supports filtering by gender, category, style, etc.
3. **Natural Language Processing**: Converts casual queries into fashion recommendations  
4. **Personalization**: Incorporates user preferences for targeted advice
5. **Contextual Understanding**: Considers occasions (brunch date, business meeting, etc.)

### Technical Architecture
- **Retrieval**: ChromaDB vector search with semantic similarity
- **Generation**: GPT-4.1-mini with fashion-specific prompts
- **Workflow**: LangGraph state management for robust processing  
- **Integration**: HuggingFace embeddings with LangChain compatibility
- **Vector Store**: ChromaDB with metadata filtering capabilities

### Performance Metrics to Consider
- **Retrieval Accuracy**: How well retrieved products match the query intent
- **Recommendation Quality**: Relevance and helpfulness of styling advice
- **User Satisfaction**: Appropriateness for specified occasions and preferences
- **System Latency**: Response time for real-world usage

In [32]:
# System Statistics and Information
print("Fashion RAG System Statistics")
print("=" * 40)

try:
    # Collection statistics
    collection_count = fashion_collection.count()
    print(f"Total fashion products in database: {collection_count}")
    
    # Sample a few products to show data structure
    if collection_count > 0:
        sample_results = fashion_collection.peek(limit=3)
        print(f"\nSample products:")
        for i, (doc, metadata) in enumerate(zip(sample_results['documents'], sample_results['metadatas'])):
            print(f"  {i+1}. {doc}")
            if metadata:
                print(f"     Categories: {metadata.get('masterCategory', 'N/A')} > {metadata.get('subCategory', 'N/A')}")
                print(f"     Color: {metadata.get('baseColour', 'N/A')}, Gender: {metadata.get('gender', 'N/A')}")
    
    # Model information
    print(f"\nModel information:")
    print(f"- HuggingFace embedding model: {model_name}")
    print(f"- SentenceTransformer model: {fashion_model.model_name if hasattr(fashion_model, 'model_name') else model_name}")
    print(f"- Vector dimensions: {len(fashion_model.encode(['test']))}")
    print(f"- Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
    print(f"- LLM: GPT-4.1-mini")
    print(f"- Vector database: ChromaDB (./fashion_vector_db)")
    print(f"- LangChain integration: {'Active' if langchain_vectorstore else '❌ Not available'}")
    
except Exception as e:
    print(f"Could not retrieve statistics: {e}")
    print("Make sure to run embeddings_generation.ipynb first to populate the database.")

Fashion RAG System Statistics
Total fashion products in database: 88838

Sample products:
  1. Turtle Check Men Navy Blue Shirt | Gender: Men | Category: Apparel - Topwear | Type: Shirts | Color: Navy Blue | Season: Fall | Usage: Casual | Year: 2011
     Categories: N/A > N/A
     Color: N/A, Gender: Men
  2. Peter England Men Party Blue Jeans | Gender: Men | Category: Apparel - Bottomwear | Type: Jeans | Color: Blue | Season: Summer | Usage: Casual | Year: 2012
     Categories: N/A > N/A
     Color: N/A, Gender: Men
  3. Titan Women Silver Watch | Gender: Women | Category: Accessories - Watches | Type: Watches | Color: Silver | Season: Winter | Usage: Casual | Year: 2016
     Categories: N/A > N/A
     Color: N/A, Gender: Women

Model information:
- HuggingFace embedding model: dejasi5459/clip-fashion-embeddings-final-10k-ft
- SentenceTransformer model: dejasi5459/clip-fashion-embeddings-final-10k-ft
- Vector dimensions: 1
- Device: CUDA
- LLM: GPT-4.1-mini
- Vector database: ChromaDB

## Summary & Next Steps

### Summary
This notebook implements a complete **LangChain-powered RAG system** for fashion recommendations that:

1. **Connects to ChromaDB** vector database with fashion product embeddings
2. **Processes natural language queries** like "What should I wear for a brunch date?"
3. **Retrieves relevant products** using semantic similarity and metadata filtering
4. **Generates personalized advice** using GPT-4.1-mini with fashion-specific prompts
5. **Supports user preferences** for gender, style, category, and price range
6. **Uses HuggingFace embeddings** with the fine-tuned model dejasi5459/clip-fashion-embeddings-final-10k-ft

### Key Components
- `HuggingFaceEmbeddings`: Integration with the fine-tuned fashion CLIP model
- `FashionProductRetriever`: Enhanced retriever with LangChain compatibility and metadata filtering
- `FashionRAGState`: State management for workflow
- `get_fashion_advice()`: Main API function for fashion queries
- `LangGraph workflow`: Retrieval → Generation pipeline
- `Chroma vector store`: LangChain-compatible interface to ChromaDB

### Integration with Assignment
This notebook completes the **LangChain RAG System** requirement from the assignment:
-  Embeds natural language queries using HuggingFace integration
-  Retrieves top-k items from vector database
-  Uses LangChain to inject items into prompts
-  Generates fashion advice with LLM
-  Supports metadata filtering and personalization
-  Integrates fine-tuned model dejasi5459/clip-fashion-embeddings-final-10k-ft

### Future Enhancements
1. **API Wrapper**: Convert to FastAPI/Flask endpoint
2. **Image Integration**: Show product images in recommendations
3. **Conversation Memory**: Multi-turn fashion consultations
4. **A/B Testing**: Compare different prompt strategies
5. **User Feedback**: Learn from recommendation ratings